<a href="https://colab.research.google.com/github/celiaferreira/Covid19_RX/blob/master/6_2_MachineLearning_DataGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 6.2. Modelos de machine learning - via Data Generation

Nesta secção são apresentados modelos tradicionais de machine learning, tendo os dados de treino sido aumentados via Data Generation. Foi usado o target com 4 labels. 

Começamos por colocar as imagens numa lista, de modo a servirem de input para os modelos.


In [ ]:
def X_to_vector(X):
  X_all_vec=[]
  X_allReshaped = X.reshape(len(X),200,200)
  print(X_allReshaped.shape)
  for i in X_allReshaped:
      img = i
      #print(img.shape)
      cols,colors = img.shape # gives dimensions for RGB array
      img_size = cols*colors
      img_1D_vector = img.reshape(img_size)
      # you can recover the orginal image with:
      img2 = img_1D_vector.reshape(cols,colors)
      X_all_vec.append(img_1D_vector)
  return X_all_vec

In [ ]:
X_train_vec=X_to_vector(X_train_aug)
X_test_vec=X_to_vector(X_test)

### 6.2.1. Random Forests
O primeiro modelo de ML a testar é uma Random Forest.

In [ ]:
Rfc = RandomForestClassifier(n_estimators = 200, criterion = "entropy", random_state = 42,
                                      min_samples_split=10,min_samples_leaf=1,
                                      max_features='sqrt', max_depth=90, bootstrap=False)

Rfc=Rfc.fit(X_train_vec, y_train)
y_pred  = Rfc.predict(X_test_vec)

In [ ]:
Accuracy_total_Rfc=Rfc.score(X_test_vec, y_test)
print('Accuracy total:',Accuracy_total_Rfc)

cm = confusion_matrix(y_test, y_pred)
conf_mat=pd.DataFrame(cm)
conf_mat.index.name='Actual'
conf_mat.columns.name='Predicted'
print(conf_mat)

print('accuracy total:', accuracy(cm))
print('precision covid:', precision(0,cm))
print('recall covid:', recall(0,cm))
print('precision total:', precision_macro_average(cm))
print('recall total:', recall_macro_average(cm))
print("label precision recall")
for label in range(len(unique_labels(y_test))):
 print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")

Accuracy total: 0.8809629959875167
Predicted    0    1    2    3
Actual                       
0          575   84    0    1
1           37  568   17   51
2            0   42  145    2
3            0   33    0  688
accuracy total: 0.8809629959875167
precision covid: 0.9395424836601307
recall covid: 0.8712121212121212
precision total: 0.885780229150267
recall total: 0.8591550733955774
label precision recall
    0     0.940  0.871
    1     0.781  0.844
    2     0.895  0.767
    3     0.927  0.954


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90       660
           1       0.78      0.84      0.81       673
           2       0.90      0.77      0.83       189
           3       0.93      0.95      0.94       721

    accuracy                           0.88      2243
   macro avg       0.89      0.86      0.87      2243
weighted avg       0.88      0.88      0.88      2243



Conclui-se que o modelo apresenta uma **precisão global de 88%**, identificando **87% dos casos COVID-19**.

O maior erro está associado à pneumonia bacteriana.

### 6.2.2 Ensamble-Adaptative Boosting
Esta secção apresenta um modelo de Adaptative Boosting. 

In [ ]:
#classificações subsequentes são ajustadas a favor das classificadas erradas por classificações anteriores.
#weak learner
#contras: perda da capacidade de generalização após o aprendizado de muitos padrões de treino (overfitting)
ada_tree = AdaBoostClassifier(n_estimators=100)
ada_tree=ada_tree.fit(X_train_vec, y_train)
Y_pred_test = ada_tree.predict(X_test_vec)

In [ ]:
Accuracy_total_AdaBoost=ada_tree.score(X_test_vec, y_test)
print('Accuracy total:',Accuracy_total_AdaBoost)

conf_mat=confusion_matrix(y_test,Y_pred_test)
conf_mat=pd.DataFrame(conf_mat)
conf_mat.index.name='Actual'
conf_mat.columns.name='Predicted'
print(conf_mat)

Accuracy total: 0.7566754418954494
Predicted    0    1    2    3
Actual                       
0          864   22    2    5
1            2  252  194  166
2            2  120  140   34
3            0   68   32  756


Este apresenta uma **precisão de 75,7%**, inferior às random forests.

### 6.2.3. KNN
Apresenta-se nesta secção outra versão de classificaçao ML: KNeighborsClassifier.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1,p=1, weights='uniform')

In [ ]:
knn_model=knn.fit(X_train_vec, y_train)
print(knn_model)
Y_pred_test = knn_model.predict(X_test_vec)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=1,
                     weights='uniform')


In [ ]:
conf_mat=confusion_matrix(y_test,knn.predict(X_test_vec))
conf_mat=pd.DataFrame(conf_mat)
conf_mat.index.name='Actual'
conf_mat.columns.name='Predicted'
print(conf_mat)

Predicted    0    1    2    3
Actual                       
0          582   77    1    0
1           46  533   47   47
2            0   26  159    4
3            1   48    6  666


In [ ]:
print('Score:',knn_model_score)

Score: 0.8649130628622381


O modelo apresenta uma precisão de **86%**, comparável com as random forests.

